# Importação

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Criação do dataset

In [3]:
# Tentando buscar dataset existente, chamado de 'data.csv'
try:
    df = pd.read_csv('data.csv')
    print(f"Dataset carregado com sucesso.")

#Criando o dataset
except FileNotFoundError:
    print(f"Dataset não encontrado. Gerando um novo dataset.")

    num_registros = 300
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2023, 12, 31)
    
    # Dicionários com os produtos para garantir consistência
    produtos_info = {
        'Laptop': {'Categoria': 'Eletronicos', 'Preco': 1800.00},
        'Mouse': {'Categoria': 'Perifericos', 'Preco': 50.00},
        'Teclado': {'Categoria': 'Perifericos', 'Preco': 120.00},
        'Monitor': {'Categoria': 'Eletronicos', 'Preco': 950.00},
        'Webcam': {'Categoria': 'Perifericos', 'Preco': 150.00},
        'Fone de Ouvido': {'Categoria': 'Eletronicos', 'Preco': 85.00},
        'Placa de Video': {'Categoria': 'Hardware', 'Preco': 3500.00},
        'SSD': {'Categoria': 'Hardware', 'Preco': 400.00},
        'Memoria RAM': {'Categoria': 'Hardware', 'Preco': 250.00},
        'Impressora': {'Categoria': 'Eletronicos', 'Preco': 600.00}
    }

    # Lista de produtos para seleção aleatória
    produtos = list(produtos_info.keys())
    
    # Diferentes formatos de data para simular inconsistência
    date_formats = ['%Y-%m-%d', '%d/%m/%Y', '%m-%d-%Y']
    
    data = []
    for i in range(1, num_registros + 1):
        
        # Gerar datas aleatórias com formatos inconsistentes
        data_venda = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
        data_formatada = data_venda.strftime(random.choice(date_formats))
        
        # Escolher produto e obter preço e categoria consistentes do dicionário e quantidade vendida
        produto_nome = random.choice(produtos)
        categoria_nome = produtos_info[produto_nome]['Categoria']
        preco_unitario = produtos_info[produto_nome]['Preco']
        quantidade_vendida = random.randint(1, 10)
        
        # Introduzir valores faltantes em aproximadamente 5% da coluna preço
        if random.random() < 0.05:
            preco_unitario = np.nan
        
        # Introduzir valores faltantes em aproximadamente 5% da coluna categoria
        if random.random() < 0.05:
            categoria_nome = np.nan
            
        data.append({
            'ID': i,
            'Data': data_formatada,
            'Produto': produto_nome,
            'Categoria': categoria_nome,
            'Quantidade': quantidade_vendida,
            'Preco': preco_unitario
        })

    df = pd.DataFrame(data)
    
    # Inserir duplicatas (aproximadamente 5% dos registros)
    num_duplicatas = int(num_registros * 0.05)
    for _ in range(num_duplicatas):
        df = pd.concat([df, df.sample(1)], ignore_index=True)
        
    print(f"Dataset simulado gerado")

Dataset não encontrado. Gerando um novo dataset.
Dataset simulado gerado


In [4]:
# Visualizando dataset
df.head()

,ID,Data,Produto,Categoria,Quantidade,Preco
0,1,19/05/2023,Teclado,Perifericos,1,120.0
1,2,2023-09-12,SSD,Hardware,4,NaN
2,3,09-08-2023,Mouse,Perifericos,10,50.0
3,4,2023-03-01,Mouse,Perifericos,4,50.0
4,5,24/04/2023,Fone de Ouvido,Eletronicos,10,85.0


In [5]:
# Visualizando informações do dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          315 non-null    int64  
 1   Data        315 non-null    object 
 2   Produto     315 non-null    object 
 3   Categoria   296 non-null    object 
 4   Quantidade  315 non-null    int64  
 5   Preco       300 non-null    float64
dtypes: float64(1), int64(2), object(3)
memory usage: 14.9+ KB


# Limpeza dos dados

In [6]:
# Copiando o dataset bruto para onde será feita a limpeza
df_clean = df.copy()

In [7]:
# Removendo duplicatas
formato_antes = df_clean.shape
df_clean.drop_duplicates(inplace=True)
formato_depois = df_clean.shape

print(f"Formato do dataset antes de remover as duplicatas: {formato_antes[0]} linhas x {formato_antes[1]} colunas.")
print(f"Formato do dataset depois de remover as duplicatas: {formato_depois[0]} linhas x {formato_depois[1]} colunas.")

Formato do dataset antes de remover as duplicatas: 315 linhas x 6 colunas.
Formato do dataset depois de remover as duplicatas: 300 linhas x 6 colunas.


In [8]:
# Resolvendo formato das colunas e valores faltantes

# Padronização da coluna 'Data'
df_clean['Data'] = np.where(
    df_clean['Data'].astype(str).str.contains('/', na=False),
    pd.to_datetime(df_clean['Data'], format='%d/%m/%Y', errors='coerce'),
    pd.to_datetime(df_clean['Data'], errors='coerce')
)

# Preço faltante sendo preenchido pela mediana dos preços de cada produto
df_clean['Preco'] = df_clean.groupby('Produto')['Preco'].transform(lambda x: x.fillna(x.median()))

# Converte a coluna 'Preco' para o tipo numérico (float) apenas para evitar qualquer tipo de erro
df_clean['Preco'] = pd.to_numeric(df_clean['Preco'], errors='coerce')

# Preencher valores faltantes da coluna 'Categoria' com base no nome do 'Produto'.
produtos_info = {
    'Laptop': 'Eletronicos',
    'Mouse': 'Perifericos',
    'Teclado': 'Perifericos',
    'Monitor': 'Eletronicos',
    'Webcam': 'Perifericos',
    'Fone de Ouvido': 'Eletronicos',
    'Placa de Video': 'Hardware',
    'SSD': 'Hardware',
    'Memoria RAM': 'Hardware',
    'Impressora': 'Eletronicos'
}
df_clean['Categoria'].fillna(df_clean['Produto'].map(produtos_info), inplace=True)

C:\Users\andre\AppData\Local\Temp\ipykernel_34416\3615059796.py:7: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  pd.to_datetime(df_clean['Data'], errors='coerce')


In [9]:
# Visualizando dataset
df_clean.head()

,ID,Data,Produto,Categoria,Quantidade,Preco
0,1,2023-05-19,Teclado,Perifericos,1,120.0
1,2,2023-09-12,SSD,Hardware,4,400.0
2,3,2023-09-08,Mouse,Perifericos,10,50.0
3,4,2023-03-01,Mouse,Perifericos,4,50.0
4,5,2023-04-24,Fone de Ouvido,Eletronicos,10,85.0


In [10]:
# Visualizando informações do dataset
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 0 to 299
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ID          300 non-null    int64         
 1   Data        300 non-null    datetime64[ns]
 2   Produto     300 non-null    object        
 3   Categoria   300 non-null    object        
 4   Quantidade  300 non-null    int64         
 5   Preco       300 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 16.4+ KB


In [11]:
# Salvando arquivo
df_clean.to_csv('data_clean.csv', index=False)
print(f"Dataset salvo como 'data_clean.csv'")

Dataset salvo como 'data_clean.csv'
